In [5]:

#Se importan módulos querys
from rutas_entrada import extraer_fecha_y_hora_desde_ruta
from rutas_entrada import *
from Query_generation_tables import *
from Query_generation_costs  import *
from Query_total_generation  import *
from Query_CMg  import *
from Query_BESS import *
# en tu entorno donde ya tienes `results`
import pickle
from pathlib import Path

#E:\Entrenamiento\20250317_Look_ahead_pruebas\PID_20250219_01\PID_20250219\Modelos
#Model Test15d Solution_auto

#Se ejecuta el módulo que genera las rutas
sol_file, sol_file_pid2, sol_file_pcp, directorio_salida, directorio_fecha, periodo_pid, ruta_prg = rutas_entrada()

from concurrent.futures import ThreadPoolExecutor, as_completed
from collections import defaultdict

# Estructura final para almacenar resultados por solución
results = defaultdict(dict)

# Diccionario de rutas de entrada por solución
solutions_files = {
    "PID1": sol_file,
    "PID2": sol_file_pid2,
    "PCP":  sol_file_pcp,
}

# Función que ejecuta UNA consulta (una función para una solución)
def run_single_query(label, path, function_name):
    is_pcp = (label == "PCP")
    st_flag = False if is_pcp else True

    try:
        if function_name == "GENTABLES":
            result = get_generation_tables(
                sol_file=path,
                tipo_solucion=label,
                directorio_salida=directorio_salida,
                directorio_fecha=directorio_fecha,
                hini=periodo_pid,
                st_schedule=st_flag
            )

        elif function_name == "GENC":
            result = get_gen_costs(
                sol_file=path,
                tipo_solucion=label,
                directorio_salida=directorio_salida,
                hini=periodo_pid,
                st_schedule=st_flag
            )

        elif function_name == "GENT":
            result = get_total_generation(
                sol_file=path,
                tipo_solucion=label,
                sol_file_pcp=path,
                directorio_salida=directorio_salida,
                directorio_fecha=directorio_fecha,
                periodo_pid=periodo_pid,
                st_schedule=st_flag,
                hini=periodo_pid,
                tx_loss=False
            )

        elif function_name == "CMG":
            result = get_cmg(
                sol_file=path,
                tipo_solucion=label,
                directorio_salida=directorio_salida,
                st_schedule=st_flag,
                hini=periodo_pid
            )

        elif function_name == "BESS":
            result = get_bess(
                sol_file=path,
                tipo_solucion=label,
                directorio_salida=directorio_salida,
                st_schedule=st_flag,
                hini=periodo_pid
            )

        else:
            raise ValueError(f"Función desconocida: {function_name}")


        print(f"[OK] {label} – {function_name}")
        return label, function_name, result

    except Exception as e:
        print(f"[ERR] {label} – {function_name}: {e}")
        return label, function_name, None

# Lista de funciones a ejecutar por cada solución
function_names = ["GENTABLES", "GENC", "GENT", "CMG", "BESS"]

# Lanzar todas las tareas (3 soluciones × 5 funciones = 15 tareas)
with ThreadPoolExecutor(max_workers=15) as executor:
    futures = [
        executor.submit(run_single_query, label, path, func)
        for label, path in solutions_files.items()
        for func in function_names
    ]

    for fut in as_completed(futures):
        label, func_name, result = fut.result()
        if result is not None:
            results[label][func_name] = result


fecha, periodo_pid = extraer_fecha_y_hora_desde_ruta(directorio_fecha)
etiqueta = f"{fecha}_{periodo_pid:02}"
directorio_salida = Path(directorio_salida)  # <- añade esta línea justo antes
output_path = directorio_salida / f"results_{etiqueta}.pkl"
output_path.write_bytes(pickle.dumps(results))


== Comparación de PRGs ==
[ERR] PID2 – CMG: _path_exists: path should be string, bytes, os.PathLike or integer, not NoneType
[ERR] PID2 – GENTABLES: _path_exists: path should be string, bytes, os.PathLike or integer, not NoneType
[ERR] PID2 – GENC: _path_exists: path should be string, bytes, os.PathLike or integer, not NoneType
[ERR] PID2 – GENT: _path_exists: path should be string, bytes, os.PathLike or integer, not NoneType
[ERR] PID2 – BESS: _path_exists: path should be string, bytes, os.PathLike or integer, not NoneType
[OK] PID1 – CMG
[OK] PCP – CMG
[OK] PID1 – GENTABLES
[OK] PID1 – BESS
[OK] PID1 – GENT
[OK] PID1 – GENC
[OK] PCP – GENT
[OK] PCP – BESS
[OK] PCP – GENTABLES
[OK] PCP – GENC


1546581

In [1]:
not True

False

In [16]:
from pathlib import Path

directorio_salida = Path(directorio_salida)  # <- añade esta línea justo antes
output_path = directorio_salida / f"results_{etiqueta}.pkl"
output_path.write_bytes(pickle.dumps(results))


2309044

In [2]:
import pickle
from pathlib import Path

archivo = Path("Resultados/results_20250701_01.pkl")
with archivo.open("rb") as f:
    results = pickle.load(f)

# Verifica qué contiene:
print(type(results))
print(results.keys())  # si es un dict


<class 'collections.defaultdict'>
dict_keys(['PID1', 'PCP'])


In [4]:
results['PID1']['GENTABLES']

(shape: (39, 49)
 ┌───────────────┬───────┬──────┬──────┬───┬───────┬──────┬───────┬───────┐
 │ Nombre_PLEXOS ┆ 1     ┆ 2    ┆ 3    ┆ … ┆ 45    ┆ 46   ┆ 47    ┆ 48    │
 │ ---           ┆ ---   ┆ ---  ┆ ---  ┆   ┆ ---   ┆ ---  ┆ ---   ┆ ---   │
 │ str           ┆ f64   ┆ f64  ┆ f64  ┆   ┆ f64   ┆ f64  ┆ f64   ┆ f64   │
 ╞═══════════════╪═══════╪══════╪══════╪═══╪═══════╪══════╪═══════╪═══════╡
 │ ANGOSTURA_U1  ┆ 0.0   ┆ 77.0 ┆ 0.0  ┆ … ┆ 0.0   ┆ 77.0 ┆ 88.1  ┆ 77.2  │
 │ ANGOSTURA_U2  ┆ 77.0  ┆ 0.0  ┆ 77.0 ┆ … ┆ 77.0  ┆ 0.0  ┆ 0.0   ┆ 0.0   │
 │ ANGOSTURA_U3  ┆ 0.0   ┆ 0.0  ┆ 0.0  ┆ … ┆ 0.0   ┆ 0.0  ┆ 0.0   ┆ 0.0   │
 │ ANTUCO_U1     ┆ 160.0 ┆ 80.0 ┆ 80.0 ┆ … ┆ 132.1 ┆ 80.6 ┆ 132.1 ┆ 132.1 │
 │ ANTUCO_U2     ┆ 132.1 ┆ 80.0 ┆ 80.0 ┆ … ┆ 160.0 ┆ 80.0 ┆ 160.0 ┆ 160.0 │
 │ …             ┆ …     ┆ …    ┆ …    ┆ … ┆ …     ┆ …    ┆ …     ┆ …     │
 │ RAPEL_U3      ┆ 58.0  ┆ 0.0  ┆ 0.0  ┆ … ┆ 0.0   ┆ 0.0  ┆ 0.0   ┆ 0.0   │
 │ RAPEL_U4      ┆ 58.0  ┆ 58.0 ┆ 58.0 ┆ … ┆ 58.0  ┆ 58.0 ┆ 58.0  ┆ 58.